# **Data Extraction**

In [171]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import sqlite3

## Assets

In [172]:
url = 'https://companiesmarketcap.com/assets-by-market-cap/'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'lxml')
table = soup.find('table', {'class':'default-table table marketcap-table dataTable'})


entries = table.find_all('tr')
entries = entries[1:]

names = []
trends =[]
pct_change = []
m_cap = []
prices = []
links = []
asset_desc = []

for entry in entries:
    name = entry.find('div', {'class':'company-name'})
    names.append(name.text.strip())

    earn = entry.find('td', {'class':'rh-sm'})
    trend = 'down' if earn.find('span', {'class': 'percentage-red'}) else 'up'
    trends.append(trend)
    
    day_ct = earn.text
    pct_change.append(day_ct)

    fintext = entry.find_all('td', {'class':'td-right'})
    cap = fintext[1].text.strip()
    m_cap.append(cap)
    
    price = fintext[2].text.strip()
    prices.append(price)
    

    url = entry.find('a')
    if url:
        link = url['href']
        full_link = f'https://companiesmarketcap.com{link}'
    else:
        full_link = None

    links.append(full_link)

for link in links:
    if link:
        r = requests.get(link)
        soup = BeautifulSoup(r.text, 'lxml')
        
        div = soup.find('div', {'class': 'col-sm-9'})
        if not div:
            div = soup.find('div', {'class': 'col-lg-4 company-description'})
        
        text = div.text.strip() if div else None
        asset_desc.append(text)
    else:
        asset_desc.append(None)      
        
# Save to dataframe        
asset_df = pd.DataFrame({'Asset': names, 'Market Cap': m_cap, 'Share Price': prices,
                   'Day Change': pct_change, 'Trend': trends,  'Description': asset_desc
                   })


# Data cleaning

# Function to convert values to billions
def convert_to_billion(value):
    if 'T' in value:
        return float(value.replace('T', '')) * 1000
    elif 'B' in value:
        return float(value.replace('B', ''))
    else:
        return float(value)

# Apply the conversion function and strip '$' and 'T'
asset_df['Market Cap'] = asset_df['Market Cap'].str.replace(r'[$,]', '', regex=True).apply(convert_to_billion)

# Rename the column
asset_df = asset_df.rename(columns={'Market Cap': 'Market Cap($B)'})
asset_df['Share Price'] = asset_df['Share Price'].str.replace(r'[$, ,]', '', regex=True).astype(float)
asset_df['As Of'] = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')
asset_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Asset           100 non-null    object 
 1   Market Cap($B)  100 non-null    float64
 2   Share Price     100 non-null    float64
 3   Day Change      100 non-null    object 
 4   Trend           100 non-null    object 
 5   Description     63 non-null     object 
 6   As Of           100 non-null    object 
dtypes: float64(2), object(5)
memory usage: 5.6+ KB


In [173]:
# Save to SQLite Database
conn = sqlite3.connect('Marketcap.db')
cursor = conn.cursor()

# Update table
asset_df.to_sql('Assets', conn, if_exists='append', index=False)

conn.commit()

print('Data successfully added to database table')

# Close the connection
conn.close()

Data successfully added to database table


## Earnings

In [174]:
# Generate a list of URLs for the first 10 pages of the most profitable companies

urls = []
base = 'https://companiesmarketcap.com/most-profitable-companies/page/'
for pg in range(1, 11):
  url = f'{base}{pg}'
  urls.append(url)


In [175]:
all_df = []
for url in urls:
  r = requests.get(url)
  soup = BeautifulSoup(r.text, 'lxml')
  table = soup.find('tbody')

  names = soup.find_all('div', {'class':'company-name'})
  names = [name.text.strip() for name in names]

  trends =[]
  earnings = soup.find_all('td', {'class':'rh-sm'})
  for earn in earnings:
      if earn.find('span', {'class':'percentage-red'}):
        trend = 'down'
      else:
        trend = 'up'
      trends.append(trend)

  day_ct = [earning.text for earning in earnings]

  tds = soup.find_all('td', {'class':'td-right'})
  earnings = []
  prices = []
  for i in range(0, len(tds), 3):
        earnings.append(tds[i+1].text.strip())
        prices.append(tds[i+2].text.strip())

  countries = soup.find_all('span',{'class':'responsive-hidden'})
  countries = countries[1:]
  countries = [country.text.strip() for country in countries]

  data = {'Company':names, 'Earnings':earnings, 'Share Price':prices, 'Pct_change':day_ct, 'Trend':trends, 'Country':countries }
  df = pd.DataFrame(data)

  # Data cleaning
  # Function to convert values to billions
  def convert_to_billion(value):
      if 'T' in value:
          return float(value.replace('T', '')) * 1000
      elif 'B' in value:
          return float(value.replace('B', ''))
      else:
          return float(value)

  # Apply the conversion function and strip '$' and 'T'
  df['Earnings'] = df['Earnings'].str.replace(r'[$,]', '', regex=True).apply(convert_to_billion)

  # Rename the column
  df = df.rename(columns={'Earnings': 'Earnings($B)'})
  df['Share Price'] = df['Share Price'].str.replace(r'[$, ,]', '', regex=True).astype(float)
  df['Pct_change'] = df['Pct_change'].str.replace(r'%', '', regex=True).astype(float)
  all_df.append(df)

# Concatenate all DataFrames into one
earnings_df = pd.concat(all_df, ignore_index=True)

# Drop duplicates 
earnings_df.drop_duplicates(subset='Company', keep='first', inplace=True)
earnings_df['As Of'] = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')

# Sort alphabetically and reset index
earnings_df = earnings_df.sort_values(by='Company')
earnings_df = earnings_df.reset_index(drop=True)
earnings_df['As Of'] = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')

earnings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Company       1000 non-null   object 
 1   Earnings($B)  1000 non-null   float64
 2   Share Price   1000 non-null   float64
 3   Pct_change    1000 non-null   float64
 4   Trend         1000 non-null   object 
 5   Country       1000 non-null   object 
 6   As Of         1000 non-null   object 
dtypes: float64(3), object(4)
memory usage: 54.8+ KB


In [176]:
earnings_df.head()

,Company,Earnings($B),Share Price,Pct_change,Trend,Country,As Of
0,3i Group,5.66,39.37,0.10,up,UK,2024-07-19 20:26
1,7-Eleven,1.89,11.43,0.50,down,Japan,2024-07-19 20:26
2,ABB,4.84,54.29,0.02,down,Switzerland,2024-07-19 20:26
3,ABN AMRO,4.05,17.69,1.10,down,Netherlands,2024-07-19 20:26
4,AES,1.46,17.17,0.53,up,USA,2024-07-19 20:26


In [177]:
# Save to SQLite Database
conn = sqlite3.connect('Marketcap.db')
cursor = conn.cursor()

# Update table
earnings_df.to_sql('Earnings', conn, if_exists='append', index=False)

conn.commit()

print('Data successfully added to database table')

# Close the connection
conn.close()

Data successfully added to database table


## Market cap

In [178]:
# Generate urls for companies with the highest market caps

base = 'https://companiesmarketcap.com/page/'
urls = []
for pg in range(1,11):
  url = f'{base}{pg}'
  urls.append(url)

In [179]:
all_df = []
for url in urls:
  r = requests.get(url)
  soup = BeautifulSoup(r.text, 'lxml')
  table = soup.find('tbody')

  names = soup.find_all('div', {'class':'company-name'})
  names = [name.text.strip() for name in names]

  trends =[]
  earnings = soup.find_all('td', {'class':'rh-sm'})
  for earn in earnings:
      if earn.find('span', {'class':'percentage-red'}):
        trend = 'down'
      else:
        trend = 'up'
      trends.append(trend)

  day_ct = [earning.text for earning in earnings]

  tds = soup.find_all('td', {'class':'td-right'})
  earnings = []
  prices = []
  for i in range(0, len(tds), 3):
        earnings.append(tds[i+1].text.strip())
        prices.append(tds[i+2].text.strip())

  countries = soup.find_all('span',{'class':'responsive-hidden'})
  countries = countries[1:]
  countries = [country.text.strip() for country in countries]

  data = {'Company':names, 'Market cap':earnings, 'Share Price':prices, 'Pct_change':day_ct, 'Trend':trends, 'Country':countries }
  df = pd.DataFrame(data)

  # Data cleaning
  # Function to convert values to billions
  def convert_to_billion(value):
      if 'T' in value:
          return float(value.replace('T', '')) * 1000
      elif 'B' in value:
          return float(value.replace('B', ''))
      else:
          return float(value)

  # Apply the conversion function and strip '$' and 'T'
  df['Market cap'] = df['Market cap'].str.replace(r'[$,]', '', regex=True).apply(convert_to_billion)

  # Rename the column
  df = df.rename(columns={'Market cap': 'Market cap($B)'})
  df['Share Price'] = df['Share Price'].str.replace(r'[$, ,]', '', regex=True).astype(float)
  df['Pct_change'] = df['Pct_change'].str.replace(r'%', '', regex=True).astype(float)
  all_df.append(df)

# Concatenate all DataFrames into one
market_df = pd.concat(all_df, ignore_index=True)
market_df.drop_duplicates(subset='Company', keep='first', inplace=True)

# Sort alphabetically and reset index
market_df = market_df.sort_values(by='Company')
market_df = market_df.reset_index(drop=True)
market_df['As Of'] = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')

market_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Company         1000 non-null   object 
 1   Market cap($B)  1000 non-null   float64
 2   Share Price     1000 non-null   float64
 3   Pct_change      1000 non-null   float64
 4   Trend           1000 non-null   object 
 5   Country         1000 non-null   object 
 6   As Of           1000 non-null   object 
dtypes: float64(3), object(4)
memory usage: 54.8+ KB


In [180]:
market_df.head()

,Company,Market cap($B),Share Price,Pct_change,Trend,Country,As Of
0,3M,57.63,104.16,0.13,up,USA,2024-07-19 20:26
1,3i Group,37.97,39.37,0.10,up,UK,2024-07-19 20:26
2,7-Eleven,29.66,11.43,0.50,down,Japan,2024-07-19 20:26
3,ABB,100.34,54.29,0.02,down,Switzerland,2024-07-19 20:26
4,ABB India,19.30,91.07,3.45,down,India,2024-07-19 20:26


In [181]:
# Save to SQLite Database
conn = sqlite3.connect('Marketcap.db')
cursor = conn.cursor()

# Update table
market_df.to_sql('Marketcap', conn, if_exists='append', index=False)

conn.commit()

print('Data successfully added to database table')

# Close the connection
conn.close()

Data successfully added to database table


## Revenue

In [182]:
base = 'https://companiesmarketcap.com/largest-companies-by-revenue/page/'
urls = []
for pg in range(1,11):
  url = f'{base}{pg}'
  urls.append(url)

In [183]:
all_df = []
for url in urls:
  r = requests.get(url)
  soup = BeautifulSoup(r.text, 'lxml')
  table = soup.find('tbody')

  names = soup.find_all('div', {'class':'company-name'})
  names = [name.text for name in names]

  trends =[]
  earnings = soup.find_all('td', {'class':'rh-sm'})
  for earn in earnings:
      if earn.find('span', {'class':'percentage-red'}):
        trend = 'down'
      else:
        trend = 'up'
      trends.append(trend)

  day_ct = [earning.text for earning in earnings]

  tds = soup.find_all('td', {'class':'td-right'})
  earnings = []
  prices = []
  for i in range(0, len(tds), 3):
        earnings.append(tds[i+1].text.strip())
        prices.append(tds[i+2].text.strip())

  countries = soup.find_all('span',{'class':'responsive-hidden'})
  countries = countries[1:]
  countries = [country.text for country in countries]

  data = {'Company':names, 'Revenue':earnings, 'Share Price':prices, 'Pct_change':day_ct, 'Trend':trends, 'Country':countries }
  df = pd.DataFrame(data)

  # Data cleaning
  # Function to convert values to billions
  def convert_to_billion(value):
      if 'T' in value:
          return float(value.replace('T', '')) * 1000
      elif 'B' in value:
          return float(value.replace('B', ''))
      else:
          return float(value)

  # Apply the conversion function and strip '$' and 'T'
  df['Revenue'] = df['Revenue'].str.replace(r'[$,]', '', regex=True).apply(convert_to_billion)

  # Rename the column
  df = df.rename(columns={'Revenue': 'Revenue($B)'})
  df['Share Price'] = df['Share Price'].str.replace(r'[$, ,]', '', regex=True).astype(float)
  df['Pct_change'] = df['Pct_change'].str.replace(r'%', '', regex=True).astype(float)
  all_df.append(df)

# Concatenate all DataFrames into one
revenue_df = pd.concat(all_df, ignore_index=True)

revenue_df.drop_duplicates(subset='Company', keep='first', inplace=True)

# Sort alphabetically and reset index
revenue_df = revenue_df.sort_values(by='Company')
revenue_df = revenue_df.reset_index(drop=True)
revenue_df['As Of'] = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')

revenue_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Company      1000 non-null   object 
 1   Revenue($B)  1000 non-null   float64
 2   Share Price  1000 non-null   float64
 3   Pct_change   1000 non-null   float64
 4   Trend        1000 non-null   object 
 5   Country      1000 non-null   object 
 6   As Of        1000 non-null   object 
dtypes: float64(3), object(4)
memory usage: 54.8+ KB


In [184]:
revenue_df.head(10)

,Company,Revenue($B),Share Price,Pct_change,Trend,Country,As Of
0,3M,32.65,104.16,0.13,up,USA,2024-07-19 20:26
1,7-Eleven,78.60,11.43,0.50,down,Japan,2024-07-19 20:26
2,A2A,18.24,2.08,1.85,down,Italy,2024-07-19 20:26
3,ABB,32.24,54.29,0.02,down,Switzerland,2024-07-19 20:26
4,AECOM,15.34,88.66,1.03,down,USA,2024-07-19 20:26
5,AEGON\n,12.01,6.20,0.56,down,Netherlands,2024-07-19 20:26
6,AES,12.51,17.17,0.53,up,USA,2024-07-19 20:26
7,AGC,13.85,33.19,1.49,down,Japan,2024-07-19 20:26
8,AGCO,14.00,102.43,0.19,up,USA,2024-07-19 20:26
9,AIA,20.69,6.93,0.73,down,Hong Kong,2024-07-19 20:26


In [185]:
# Save to SQLite Database
conn = sqlite3.connect('Marketcap.db')
cursor = conn.cursor()

# Update table
revenue_df.to_sql('Revenue', conn, if_exists='append', index=False)

conn.commit()

print('Data successfully added to database table')

# Close the connection
conn.close()

Data successfully added to database table


## Employee size

In [186]:
base = 'https://companiesmarketcap.com/largest-companies-by-number-of-employees/page/'
urls = []
for pg in range(1,11):
  url = f'{base}{pg}'
  urls.append(url)

In [187]:
all_df = []
for url in urls:
  r = requests.get(url)
  soup = BeautifulSoup(r.text, 'lxml')
  table = soup.find('tbody')

  names = soup.find_all('div', {'class':'company-name'})
  names = [name.text.strip() for name in names]

  trends =[]
  earnings = soup.find_all('td', {'class':'rh-sm'})
  for earn in earnings:
      if earn.find('span', {'class':'percentage-red'}):
        trend = 'down'
      else:
        trend = 'up'
      trends.append(trend)

  day_ct = [earning.text for earning in earnings]

  tds = soup.find_all('td', {'class':'td-right'})
  employees = []
  prices = []
  for i in range(0, len(tds), 3):
        employees.append(tds[i+1].text.strip())
        prices.append(tds[i+2].text.strip())

  countries = soup.find_all('span',{'class':'responsive-hidden'})
  countries = countries[1:]
  countries = [country.text.strip() for country in countries]

  data = {'Company':names, 'Employees':employees, 'Share Price':prices, 'Pct_change':day_ct, 'Trend':trends, 'Country':countries }
  df = pd.DataFrame(data)

  # Data cleaning
  df['Employees'] = df['Employees'].str.replace(r'[,]', '', regex=True).astype(int)
  df['Share Price'] = df['Share Price'].str.replace(r'[$, ,]', '', regex=True).astype(float)
  df['Pct_change'] = df['Pct_change'].str.replace(r'%', '', regex=True).astype(float)
  all_df.append(df)

# Concatenate all DataFrames into one
employee_df = pd.concat(all_df, ignore_index=True)

employee_df.drop_duplicates(subset='Company', keep='first', inplace=True)

# Sort alphabetically and reset index
employee_df = employee_df.sort_values(by='Company')
employee_df = employee_df.reset_index(drop=True)
employee_df['As Of'] = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')

employee_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Company      1000 non-null   object 
 1   Employees    1000 non-null   int64  
 2   Share Price  1000 non-null   float64
 3   Pct_change   1000 non-null   float64
 4   Trend        1000 non-null   object 
 5   Country      1000 non-null   object 
 6   As Of        1000 non-null   object 
dtypes: float64(2), int64(1), object(4)
memory usage: 54.8+ KB


In [188]:
employee_df.head(10)

,Company,Employees,Share Price,Pct_change,Trend,Country,As Of
0,3M,85000,104.16,0.13,up,USA,2024-07-19 20:26
1,7-Eleven,84154,11.43,0.50,down,Japan,2024-07-19 20:26
2,ABB,108700,54.29,0.02,down,Switzerland,2024-07-19 20:26
3,ABM Industries,100000,53.75,0.02,down,USA,2024-07-19 20:26
4,AECOM,52000,88.66,1.03,down,USA,2024-07-19 20:26
5,AGC,56724,33.19,1.49,down,Japan,2024-07-19 20:26
6,AGCO,27900,102.43,0.19,up,USA,2024-07-19 20:26
7,AIER Eye Hospital,32326,1.49,0.55,down,China,2024-07-19 20:26
8,ALSOK,39039,6.09,0.72,up,Japan,2024-07-19 20:26
9,ALTEN,57000,111.69,5.96,down,France,2024-07-19 20:26


In [189]:
# Save to SQLite Database
conn = sqlite3.connect('Marketcap.db')
cursor = conn.cursor()

# Update table
employee_df.to_sql('Employees', conn, if_exists='append', index=False)

conn.commit()

print('Data successfully added to database table')

# Close the connection
conn.close()

Data successfully added to database table


# **Transformation**

In [190]:
# Select only the relevant columns from each DataFrame
earns = earnings_df[['Company', 'Earnings($B)', 'Country', 'Trend']]
cap = market_df[['Company', 'Market cap($B)']]
revenue = revenue_df[['Company', 'Revenue($B)']]
employee = employee_df[['Company', 'Employees']]

# Merge the DataFrames on the 'Company' column
merged_df = pd.merge(earns, cap, on='Company')
merged_df = pd.merge(merged_df, revenue, on='Company')
merged_df = pd.merge(merged_df, employee, on='Company')

# Reorder columns
cols = merged_df.columns.tolist()
cols.remove('Trend')
cols.remove('Country')  
cols.append('Country')
cols.append('Trend')
merged_df = merged_df[cols] 
merged_df['As Of'] = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')

merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335 entries, 0 to 334
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Company         335 non-null    object 
 1   Earnings($B)    335 non-null    float64
 2   Market cap($B)  335 non-null    float64
 3   Revenue($B)     335 non-null    float64
 4   Employees       335 non-null    int64  
 5   Country         335 non-null    object 
 6   Trend           335 non-null    object 
 7   As Of           335 non-null    object 
dtypes: float64(3), int64(1), object(4)
memory usage: 21.1+ KB


In [191]:
merged_df.head(10)

,Company,Earnings($B),Market cap($B),Revenue($B),Employees,Country,Trend,As Of
0,7-Eleven,1.89,29.66,78.60,84154,Japan,down,2024-07-19 20:26
1,ABB,4.84,100.34,32.24,108700,Switzerland,down,2024-07-19 20:26
2,ANZ Bank,6.48,59.68,13.05,40342,Australia,down,2024-07-19 20:26
3,ASML,9.12,361.39,28.26,40940,Netherlands,down,2024-07-19 20:26
4,AT&T,18.95,137.27,122.31,148290,USA,down,2024-07-19 20:26
5,AXA,7.36,78.30,134.68,94705,France,down,2024-07-19 20:26
6,AbbVie,7.53,302.91,54.40,50000,USA,up,2024-07-19 20:26
7,Abbott Laboratories,6.53,176.62,40.32,114000,USA,up,2024-07-19 20:26
8,Accenture,9.37,206.08,64.47,750000,Ireland,up,2024-07-19 20:26
9,Adobe,6.57,244.22,20.42,29945,USA,down,2024-07-19 20:26


In [192]:
# Save to SQLite Database
conn = sqlite3.connect('Marketcap.db')
cursor = conn.cursor()

# Update table
merged_df.to_sql('CompanyMerge', conn, if_exists='append', index=False)

conn.commit()

print('Data successfully added to database table')

# Close the connection
conn.close()

Data successfully added to database table


# **Insight Extraction**

### Explore top companies

In [193]:
metrics = cols[1:-2]

print(f'Sample metrics include:')
for i, metric in enumerate(metrics, start=1):
    print(f"{i}. {metric}")

Sample metrics include:
1. Earnings($B)
2. Market cap($B)
3. Revenue($B)
4. Employees


In [194]:
# Sort and display dataframe
sort_key = input('Enter a metric to sort by:')
choice = metrics[int(sort_key)-1]
sort_df = merged_df.sort_values(by=choice, ascending=False)
sort_df = sort_df.reset_index(drop=True)
print(f'\nDisplaying top companies by {choice}:\n')
sort_df.head(20)


Displaying top companies by Revenue($B):



,Company,Earnings($B),Market cap($B),Revenue($B),Employees,Country,Trend,As Of
0,Walmart,28.92,569.44,657.33,2100000,USA,down,2024-07-19 20:26
1,Amazon,49.42,1901.00,590.74,1525000,USA,down,2024-07-19 20:26
2,Saudi Aramco,230.48,1836.00,489.43,70000,S. Arabia,up,2024-07-19 20:26
3,Sinopec,15.55,102.24,473.53,374791,China,down,2024-07-19 20:26
4,PetroChina,36.82,241.77,430.65,398440,China,down,2024-07-19 20:26
5,Apple,120.00,3437.00,381.62,150000,USA,down,2024-07-19 20:26
6,UnitedHealth,21.78,520.62,379.48,440000,USA,up,2024-07-19 20:26
7,CVS Health,12.65,74.74,360.93,219000,USA,down,2024-07-19 20:26
8,Volkswagen,23.71,59.70,348.13,650951,Germany,down,2024-07-19 20:26
9,Exxon Mobil,48.34,524.58,331.46,62000,USA,down,2024-07-19 20:26


### Explore trends

In [200]:
trend = input('Select a trend; up or down:')

filtered_df = merged_df[merged_df['Trend'] == trend]
df_size = len(filtered_df)

if trend == 'up':
  if df_size > 0:
    print(f"There are {len(filtered_df)} Companies with increasing trends today:")
    print(filtered_df['Company'].head(30))
  else:
    print("No companies found with increasing trends.")
elif trend =='down':
  if df_size > 0:
    print(f"There are {len(filtered_df)} Companies with decreasing trends today:")
    print(filtered_df['Company'].head(30))
  else:
    print("No companies found with decreasing trends.")
else:
  print("Invalid input. Please try again.") 


There are 248 Companies with decreasing trends today:
                                 Company  Earnings($B)  Market cap($B)  \
0                               7-Eleven          1.89           29.66   
1                                    ABB          4.84          100.34   
2                               ANZ Bank          6.48           59.68   
3                                   ASML          9.12          361.39   
4                                   AT&T         18.95          137.27   
5                                    AXA          7.36           78.30   
9                                  Adobe          6.57          244.22   
10            Agricultural Bank of China         42.33          220.31   
11                        Ahold Delhaize          2.44           30.02   
12                           Air Liquide          4.59          102.28   
14                               Alibaba         17.34          181.90   
15                               Allianz         13.69    